import necessary packages

In [ ]:
from imutils import build_montages
import numpy as np
import cv2
import pickle
from matplotlib import pyplot as plt
import random

load our dataset

In [ ]:
import tensorflow
mnist = tensorflow.keras.datasets.mnist
(x_train,y_train), (x_valid, y_valid) = mnist.load_data()

configs

In [ ]:
import os

IMG_SHAPE = (28, 28,1)
BATCH_SIZE = 20
EPOCHS = 10
BASE_OUTPUT = "output"
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "siamese_model"])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])

create simple network it depends on our siamese model

In [ ]:
# import neccessary packages
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Flatten, BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
import tensorflow

def build_siamese_model(inputShape, embeddingDim=70):
  inputs = Input(inputShape)
  x = Conv2D(64, (7, 7), activation='relu', padding='valid')(inputs)

  x = Conv2D(64, (5, 5), activation='relu', padding='valid')(x)

  x = Conv2D(128, (5, 5), activation='relu', padding='valid')(x)

  x = Conv2D(128, (5, 5), activation='relu', padding='valid')(x)

  x = Conv2D(128, (5, 5), activation='relu', padding='valid')(x)

  x = Conv2D(128, (3, 3), activation='relu', padding='valid')(x)

  x = Conv2D(256, (3, 3), activation='relu', padding='valid')(x)

  pooledOutput = GlobalAveragePooling2D()(x)
  outputs = Dense(embeddingDim)(pooledOutput)
  model = Model(inputs, outputs)
	# return the model to the calling function
  model.summary()
  return model


utils file

In [ ]:
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
def euclidean_distance(vectors):
	# unpack the vectors into separate lists
	(featsA, featsB) = vectors
	# compute the sum of squared distances between the vectors
	sumSquared = K.sum(K.square(featsA - featsB), axis=1,
		keepdims=True)
	# return the euclidean distance between the vectors
	return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def plot_training(H, plotPath):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(H.history["loss"], label="train_loss")
	plt.plot(H.history["val_loss"], label="val_loss")
	plt.plot(H.history["accuracy"], label="train_acc")
	plt.plot(H.history["val_accuracy"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)

load model weights

In [ ]:
from tensorflow.keras.layers import Lambda
from google.colab import drive
#drive.mount('/content/drive')
imgA = Input(shape=IMG_SHAPE)
imgB = Input(shape=IMG_SHAPE)
featureExtractor = build_siamese_model(IMG_SHAPE)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)
model.load_weights('./siamese_network.h5') # base on where siamese network saved

load features of mnist images

In [ ]:
train_featured = np.load('./train_features.npy')
valid_featured = np.load('./valid_features.npy')

model that predcit similarity based on features

In [ ]:
imgA = Input(shape=(70,))
imgB = Input(shape=(70,))
distance = Lambda(euclidean_distance)([imgA, imgB])
outputs = model.layers[-1](distance)
new_model = Model(inputs=[imgA, imgB], outputs=outputs)

if you have deleted some train data also delete it from train features

In [ ]:
index_to_delete = []

index_to_delete.sort(reverse=True)
train_featured = np.delete(train_featured,index_to_delete,axis=0)
y_train = np.delete(y_train,index_to_delete,axis=0)
x_train = np.delete(x_train,index_to_delete,axis=0)

get similarity

In [ ]:
from tensorflow.keras.models import load_model
from imutils.paths import list_images
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2

NUMBER_OF_SIMILARS = train_featured.shape[0]
# no error
WRONG_PREDICTED_VALIDS = [321, 326, 447, 449, 582, 619, 625, 646, 659, 674, 720, 726, 947, 1014, 1114, 1232, 1242, 1260, 1299, 1344, 1459, 1737, 1901, 1903, 2033, 2035, 2109, 2129, 2130, 2135, 2225, 2454, 2462, 2488, 2597, 2654, 2939, 2953, 3225, 3422, 3558, 3762, 3985, 4065, 4078, 4176, 4289, 4497, 4504, 4536, 4699, 4761, 4783, 4823, 4860, 5165, 5937, 5955, 6101, 6173, 6400, 6560, 6571, 6576, 6597, 6651, 8094, 8287, 8527, 9123, 9638, 9642, 9664, 9692, 9693, 9729, 9811, 9839]
def get_max_similars_to_img(img,n):
  valid_images = []
  no_images = train_featured.shape[0]
  mypair1 = np.ndarray(shape=(no_images, 70))
  mypair1[:] = valid_featured[img]
  mypair2 = train_featured[0:no_images]
  pred = new_model.predict([mypair2,mypair1], verbose=False)
  pred = pred.reshape(-1)
  ind = np.argsort(pred)[::-1][0:n]
  similarities = pred[ind]
  return (ind,similarities)
def matrix_of_similars():
  result = []
  for img in WRONG_PREDICTED_VALIDS:
    result.append(get_max_similars_to_img(img,NUMBER_OF_SIMILARS))
  return result
similars_matrix = matrix_of_similars()

In [ ]:
#PARAMS
NUM_CLASSES = 10
CONFIDENT_PERCENTAGE = 0.9
MIN_REPEAT = 1
SIMILARS = 500
def clean_data():
  predicted_wrongs = {}
  counter = 0
  for row in similars_matrix:
    indexes = row[0][:SIMILARS]
    similarities = row[1]
    for index in indexes:
      try:
        predicted_wrongs[index][y_valid[WRONG_PREDICTED_VALIDS[counter]]] += 1
      except:
         predicted_wrongs[index] = [0 for i in range(NUM_CLASSES)]
         predicted_wrongs[index][y_valid[WRONG_PREDICTED_VALIDS[counter]]] += 1
    counter += 1
  new_predicted_wrongs = {}
  for k,v in predicted_wrongs.items():
    if sum(v) >= MIN_REPEAT:
      new_predicted_wrongs[k] = [i/sum(v) for i in v]
  for k,v in new_predicted_wrongs.items():
    new_predicted_wrongs[k] = [max(v), v.index(max(v))]
  new_predicted_wrongs = dict(sorted(new_predicted_wrongs.items(), key=lambda item: item[1][0], reverse=True))
  final_result = []
  for k,v in new_predicted_wrongs.items():
    if v[0]> CONFIDENT_PERCENTAGE and y_train[k] != v[1]:
      final_result.append(k)
  print(f'find {len(final_result)} items')
  print(final_result)
  return final_result
results = clean_data()

plot top 10 detection

In [ ]:
counter = 0
indexes = results[:10]
print(indexes)
import matplotlib.pyplot as plt
def plot_images(indexes,title=None):
    images = x_train[indexes]
    labels = y_train[indexes]
    fig,axs = plt.subplots(nrows=2 ,ncols=5)
    fig.suptitle(f'top 10 detection {title}')
    counter = 0
   # fig.set_title('top 10 clean lab')
    for i in range(2):
        for j in range(5):
            axs[i][j].imshow(images[counter])
            axs[i][j].axis('off')
            axs[i][j].set_title((labels[counter]))
            counter+=1
        #axs[i].title(labels[i])
    plt.show()
plot_images(indexes,f'our solution')